In [136]:
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.animation as animation

from keras import models;

In [137]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(matrix):
    n = matrix.shape[1]
    cos = lambda a, b: dot(a, b) / (norm(a) * norm(b))
    
    new_matrix = np.zeros([n, n])

    for i in range(n):
        for j in range(i+1):
            new_matrix[i, j] = 1 - cos(matrix[:, i], matrix[:, j])
            new_matrix[j, i] = new_matrix[i, j]

    return new_matrix

def model_cos(model, layer_num):
    layers = model.layers
    weight_matrix = layers[layer_num].get_weights()[0]
    return cosine_similarity(weight_matrix)

In [138]:
from gtda.homology import VietorisRipsPersistence
VR = VietorisRipsPersistence(metric="precomputed")

tries = 480
cuts = [0]

In [142]:
series_number = 1004242

get_model_directory = lambda series, cut: f"models/{series}_series/{cut}_cut/"
get_model_name = lambda number: f"m_{number}.h5"

def get_model(cut_num, number):
    model_name = get_model_directory(series_number, cut_num) + get_model_name(number)
    return models.load_model(model_name, compile = False) 

In [143]:
def get_model_cos (cut, num):
    model = get_model(cut, num)
    return [model_cos(model, i) for i in range(1, len(model.layers))]

def analyse_model(cut, num):
    return VR.fit_transform(get_model_cos(cut, num)) 

def get_all_diagrams_from(cut):
    all = [analyse_model(cut, num) for num in range(1, tries+1)]
    return [[all[i][layer] for i in range(tries)] for layer in range(3)] 

In [144]:
def get_diagrams_overall(): 
    return [get_all_diagrams_from(c) for c in cuts]

all_diagrams = get_diagrams_overall()

In [145]:
from gtda.plotting import plot_diagram

In [146]:
import plotly.graph_objects as go

In [147]:
len(all_diagrams[0][0])

480

In [157]:
import plotly.graph_objects as go

# works only with dimensions up to 2
layer = 1

fig = go.Figure()

fig = go.Figure()
fig.add_trace(go.Scatter(
    visible = True,
    x=[0, 1],
    y=[0, 1],
    mode="lines",
    line={"dash": "dash", "width": 1, "color": "black"},
    showlegend=False,
    hoverinfo="none"
    ))

# Add traces, one for each slider step
dims = [0,1]
for step in range(tries//4):
    diagram = all_diagrams[0][layer][step * 4]
    diagram = diagram[diagram[:, 0] != diagram[:, 1]]

    for dim in dims:
        sub_diagram = diagram[diagram[:, 2] == dim]
        
        fig.add_trace(
            go.Scatter(visible = False, x = sub_diagram[:, 0], y = sub_diagram[:, 1], 
                    mode = "markers", 
                    name = f"step {4 * step}, H{dim}")
        )

# Create and add slider
steps = []

for i in range(1, len(fig.data), 2):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    # print(i)
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+1] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 16},
    steps=steps
)]

fig.update_layout(
    sliders=sliders, 
    xaxis = dict(range=[0,1]), 
    yaxis = dict(range=[0,1]),
    width = 600, 
    height = 600
)

fig.show()